In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.coordinates as coord
import astropy.units as u
from SelfCalGroupFinder.py.pyutils import *
import SelfCalGroupFinder.py.plotting as pp
from SelfCalGroupFinder.py.dataloc import *
import SelfCalGroupFinder.py.catalog_definitions as cat
from SelfCalGroupFinder.py.groupcatalog import deserialize, serialize, GroupCatalog, SDSSGroupCatalog, mstar_vmax_weighted, add_halo_columns
import SelfCalGroupFinder.py.groupcatalog as gc
import emcee

%load_ext autoreload
%autoreload 2

In [ ]:
DPI = 150
pp.DPI = DPI

After the group finder is run, this notebook is used to post pp.process the results, generating plots and such for analysis.


## Loading existing datasets 

In [ ]:
mxxl_all=deserialize(cat.mxxl_all)
mxxl_fiberonly=deserialize(cat.mxxl_fiberonly)
mxxl_nn=deserialize(cat.mxxl_nn)
mxxl_simple_4=deserialize(cat.mxxl_simple_4)

In [ ]:
sdss_vanilla = deserialize(cat.sdss_vanilla_v2)
sdss_colors = deserialize(cat.sdss_colors_v2)
sdss_colors_chi = deserialize(cat.sdss_colors_chi_v2)
cat.sdss_published.postprocess()
sdss_published = cat.sdss_published # It really is ~exactly sdss_colors_chi, which is great news for reproducibility
sdss_vanilla_old = deserialize(cat.sdss_vanilla)

In [ ]:
fiberonly_BGS=deserialize(cat.bgs_fiberonly)
fiberonly_1pass_BGS=deserialize(cat.bgs_fiberonly_1pass)
nn_BGS=deserialize(cat.bgs_nn)
nn_BGS_sdsslike=deserialize(cat.bgs_nn_sdsslike)
simple2_BGS=deserialize(cat.bgs_simple_2)
simple2_BGS_c=deserialize(cat.bgs_simple_2_c)
simple4_BGS=deserialize(cat.bgs_simple_4)
simple4_BGS_old=deserialize(cat.bgs_simple_4_old)
simple4_BGS_c=deserialize(cat.bgs_simple_4_c)
bgs_simple_4_no_sdss=deserialize(cat.bgs_simple_4_no_sdss)
bgs_simple_4_4p = deserialize(cat.bgs_simple_4_4p)
bgs_simple_4_1pass = deserialize(cat.bgs_simple_4_1pass)

In [ ]:
bgs_y3_simple_4 = deserialize(cat.bgs_y3_simple_4)
bgs_y3_simple_4_4p = deserialize(cat.bgs_y3_simple_4_4p)

In [ ]:
bgs_sv3_simple_4_10p = deserialize(cat.bgs_sv3_simple_4_10p)
bgs_sv3_simple_4_9p = deserialize(cat.bgs_sv3_simple_4_9p)
bgs_sv3_simple_4_8p = deserialize(cat.bgs_sv3_simple_4_8p)
bgs_sv3_simple_4_7p = deserialize(cat.bgs_sv3_simple_4_7p)
bgs_sv3_simple_4_6p = deserialize(cat.bgs_sv3_simple_4_6p)
bgs_sv3_simple_4_5p = deserialize(cat.bgs_sv3_simple_4_5p)
bgs_sv3_simple_4_4p = deserialize(cat.bgs_sv3_simple_4_4p)
bgs_sv3_simple_4_3p = deserialize(cat.bgs_sv3_simple_4_3p)
bgs_sv3_simple_4_2p = deserialize(cat.bgs_sv3_simple_4_2p)
bgs_sv3_simple_4_1p = deserialize(cat.bgs_sv3_simple_4_1p)

bgs_sv3_fiberonly_10p = deserialize(cat.bgs_sv3_fiberonly_10p)

bgs_sv3_nn_6p = deserialize(cat.bgs_sv3_nn_6p)

In [ ]:
all_u = deserialize(cat.uchuu_all)

In [ ]:
run = 1
path = f'/mount/sirocco1/imw2293/GROUP_CAT/MCMC/mcmc_{run}/mcmc_{run}.dat'
reader = emcee.backends.HDFBackend(path, read_only=True)
sdss_colors_mine = SDSSGroupCatalog.from_MCMC(reader, "SDSS Colors Mine", SDSS_v2_DAT_FILE, SDSS_v2_GALPROPS_FILE)
sdss_colors_mine.run_group_finder(popmock=True)
sdss_colors_mine.run_corrfunc()
sdss_colors_mine.postprocess()
serialize(sdss_colors_mine)

## Publishing / Sharing

In [ ]:
# Make a call like this to write a csv for sharing on NERSC
simple4_BGS.write_sharable_output_file()
bgs_simple_4_1pass.write_sharable_output_file()

## View Plots

In [ ]:
# Investigate changes in halo mass function from wcen
m1=np.log10(sdss_vanilla.all_data['M_halo'])
m2=np.log10(sdss_colors.all_data['M_halo'])
m3=np.log10(sdss_colors_chi.all_data['M_halo'])

# bin m1,m2,m3 the same way
n_bins = 20
bins = np.linspace(10.8, 15.0, n_bins)
d1 = np.digitize(m1, bins)
d2 = np.digitize(m2, bins)
d3 = np.digitize(m3, bins)

# count the number of galaxies in each bin
n1 = np.array([np.sum(d1==i) for i in range(1, n_bins+1)])
n2 = np.array([np.sum(d2==i) for i in range(1, n_bins+1)])
n3 = np.array([np.sum(d3==i) for i in range(1, n_bins+1)])

# Do the same but for log10(counts)
n1 = np.log10(n1)
n2 = np.log10(n2)
n3 = np.log10(n3)
print(n1,n2,3)

# Log difference
p1 = np.abs(n1-n2)
p2 = np.abs(n1-n3)

plt.plot(bins, p1, label='SDSS Colors vs Vanilla')
plt.plot(bins, p2, label='SDSS Colors+Chi vs Vanilla')

plt.xlabel('log10(M_halo)')
plt.ylabel('Log10 Difference in Counts')
plt.legend()

In [ ]:
# Primary Comparison
bgs_y3_simple_4.color = 'darkorange'
bgs_sv3_simple_4_7p.color = 'red'
pp.plots(simple4_BGS, bgs_y3_simple_4,gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_10p), gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_7p), sdss_vanilla)


In [ ]:
pp.single_plots(bgs_sv3_simple_4_10p)

In [ ]:
pp.plots(simple4_BGS, simple4_BGS_old)

In [ ]:
# BGS Completeness and NN/Other z assignment stats
def completeness_stats(d: GroupCatalog):
    name = d.name.replace("Simple v4 BGS sv3", "SV3")
    print(f"{name} completeness: {np.sum(d.all_data.z_assigned_flag == 0) / len(d.all_data.z_assigned_flag):.1%}")
    print(f"  NN/Other ratio: {np.sum(d.all_data.z_assigned_flag == 1) / np.sum(d.all_data.z_assigned_flag != 0):.1%}")

for d in [simple4_BGS, simple4_BGS_old, bgs_simple_4_no_sdss, bgs_simple_4_4p, bgs_y3_simple_4, bgs_y3_simple_4_4p]:
    completeness_stats(d)

for d in [bgs_sv3_simple_4_10p, bgs_sv3_simple_4_9p, bgs_sv3_simple_4_8p, bgs_sv3_simple_4_7p, bgs_sv3_simple_4_6p, bgs_sv3_simple_4_5p, bgs_sv3_simple_4_4p, bgs_sv3_simple_4_3p, bgs_sv3_simple_4_2p, bgs_sv3_simple_4_1p]:
    completeness_stats(d)


In [ ]:
# SV3 PLOTS (FULL 10 pass FOOTPRINT)
bgs_sv3_simple_4_7p.color = 'red'

pp.LEGENDS_ON = False
pp.plots(bgs_sv3_simple_4_10p, bgs_sv3_simple_4_9p, bgs_sv3_simple_4_8p, bgs_sv3_simple_4_7p, bgs_sv3_simple_4_6p, bgs_sv3_simple_4_5p, bgs_sv3_simple_4_4p, bgs_sv3_simple_4_3p, bgs_sv3_simple_4_2p, bgs_sv3_simple_4_1p)
pp.LEGENDS_ON = True

pp.completeness_comparison(bgs_sv3_simple_4_10p, bgs_sv3_simple_4_9p, bgs_sv3_simple_4_8p, bgs_sv3_simple_4_7p, bgs_sv3_simple_4_6p, bgs_sv3_simple_4_5p, bgs_sv3_simple_4_4p, bgs_sv3_simple_4_3p, bgs_sv3_simple_4_2p, bgs_sv3_simple_4_1p)


In [ ]:
pp.z_assigned_fraction(bgs_sv3_simple_4_10p, bgs_sv3_simple_4_9p, bgs_sv3_simple_4_8p, bgs_sv3_simple_4_7p, bgs_sv3_simple_4_6p, bgs_sv3_simple_4_5p, bgs_sv3_simple_4_4p, bgs_sv3_simple_4_3p, bgs_sv3_simple_4_2p, bgs_sv3_simple_4_1p)

In [ ]:
# SV3 PLOTS (USING CENTERS ONLY)
bgs_sv3_simple_4_7p.color = 'red'

pp.LEGENDS_ON = False
pp.plots(
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_10p),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_9p),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_8p),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_7p),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_6p),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_5p),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_4p),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_3p),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_2p),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_1p)
)    
pp.LEGENDS_ON = True

In [ ]:
# SV3 with pure NN
bgs_sv3_nn_6p.color = [0.6, 0.6, 0.0]
pp.plots(bgs_sv3_simple_4_10p, bgs_sv3_simple_4_6p, bgs_sv3_nn_6p)

In [ ]:
# View plots for my SDSS results from my MCMC chains
to_compare = sdss_vanilla
to_compare.run_group_finder(popmock=True)
to_compare.run_corrfunc()
to_compare.postprocess()

pp.proj_clustering_plot(sdss_vanilla)
pp.lsat_data_compare_plot(sdss_vanilla)


#sdss_colors_chi.color = 'r'
#pp.plots(sdss_colors_mine, sdss_colors, sdss_colors_chi)

In [ ]:
#simple4_BGS.name = 'DESI BGS Y1'
pp.plots(simple4_BGS, sdss_vanilla, sdss_colors, sdss_colors_chi)
#pp.plots(cat.sdss_published, sdss_colors_chi)

In [ ]:
# Why doesn't mstar missing % exactly match z_assigned_flag? 
# Probably redshift failures. Still have a spectra so still have mstar
print(np.sum(np.isnan(simple4_BGS.all_data.mstar)) / len(simple4_BGS.all_data.mstar))
print(np.sum(simple4_BGS.all_data.z_assigned_flag != 0) / len(simple4_BGS.all_data.z_assigned_flag))

In [ ]:
pp.compare_fsat_color_split(sdss_vanilla_old, sdss_vanilla, project_percent=0.52)


In [ ]:
bgs_simple_4_1pass.color = 'r'
pp.plots(simple4_BGS, bgs_simple_4_1pass)

In [ ]:
pp.compare_fsat_color_split(bgs_sv3_simple_4_10p, sdss_vanilla)


In [ ]:
#pp.compare_fsat_color_split(all, simple_2)
#pp.compare_fsat_color_split(all, simple_4)
#pp.compare_fsat_color_split(sdss_vanilla, simple4_BGS)
pp.compare_fsat_color_split(bgs_sv3_simple_4_10p, simple4_BGS)
pp.compare_fsat_color_split(bgs_sv3_simple_4_10p, bgs_sv3_fiberonly_10p)
#pp.compare_fsat_color_split(bgs_sv3_10p, bgs_sv3_10p_all)
#pp.compare_fsat_color_split(sdss_vanilla, bgs_simple_4_1pass)


In [ ]:
pp.qf_cen_plot(simple4_BGS)
pp.qf_cen_plot(sdss_published)

In [ ]:
pp.fsat_by_z_bins(simple4_BGS, z_bins=np.array([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 1.0]), show_plots=True)
#pp.fsat_by_z_bins(mxxl_simple_4, z_bins=np.array([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 1.0]), show_plots=False, aggregation=pp.fsat_truth_vmax_weighted)


In [ ]:
pp.fsat_by_z_bins(simple4_BGS, z_bins=np.array([0.0, 0.2, 1.0]))


In [ ]:
pp.plots(mxxl_all, mxxl_fiberonly, mxxl_nn, mxxl_simple_4)


In [ ]:
pp.plots(fiberonly_BGS, nn_BGS, simple4_BGS)


In [ ]:
simple4_BGS.name = 'Our Algorithm BGS Y1'

In [ ]:
pp.plots(bgs_sv3_simple_4_10p, bgs_sv3_fiberonly_10p)


In [ ]:
# Print out biggest group size
for dataset in [simple4_BGS, simple2_BGS, fiberonly_BGS, fiberonly_1pass_BGS, sdss_vanilla]:
    print(dataset.name)
    print(dataset.all_data.groupby('igrp').size().max())


In [ ]:
# SDSS Examine Bimodality

z=sdss_vanilla.all_data['z']
gmr=sdss_vanilla.all_data['Mag_g'] - sdss_vanilla.all_data['Mag_r']
junk=plt.hist(gmr, bins=np.linspace(-1,3,300), alpha=0.4)
#junk=plt.hist(k_correct(sdss_vanilla.all_data['Mag_g'], z, gmr, band='g')  - k_correct(sdss_vanilla.all_data['Mag_r'], z, gmr, band='r'), bins=500, alpha=0.4)
junk=plt.hist(sdss_vanilla.all_data['Dn4000'], bins=np.linspace(0,4,300), alpha=0.4)
plt.xlim(-1, 3)

## Make single group CSV for legacysurvey.org/viewer visualization

In [ ]:
df = pd.read_csv(OUTPUT_FOLDER + 'NERSC_BGS_1pass_v1.out')
centrals_of_big_groups = df['N_sat'] > 0
group_ids = df.loc[centrals_of_big_groups].igrp.unique()

In [ ]:
df[df.igrp == 1644058]

In [ ]:
print(group_ids[0:10])

In [ ]:
#for i in group_ids[0:10]:
for i in [1644058, 1644051]:
    #df.loc[df.igrp == i, ['RA', 'Dec']].to_csv(OUTPUT_FOLDER + f'group{i}.csv', index=False)
    print(df.loc[df.igrp == i, ['RA', 'Dec', 'z', 'z_assigned_flag']])

## BGS and SDSS Target Overlap Analysis

TODO: need to use a version of SDSS data that doesn't have nearest-neighbor assigned redshifts in it!

In [ ]:
pd.options.mode.copy_on_write = True

# For this comparison, use pure NN BGS 
bgs_to_use = simple4_BGS.all_data
lost_bgs = bgs_to_use.loc[bgs_to_use['z_assigned_flag'] != 0]
sdss_cat = sdss_vanilla.all_data

catalog = coord.SkyCoord(ra=sdss_cat.RA.to_numpy()*u.degree, dec=sdss_cat.Dec.to_numpy()*u.degree, frame='icrs')
to_match = coord.SkyCoord(ra=lost_bgs.RA.to_numpy()*u.degree, dec=lost_bgs.Dec.to_numpy()*u.degree, frame='icrs')

idx, d2d, d3d = coord.match_coordinates_sky(to_match, catalog, nthneighbor=1, storekdtree=False)

# if angular distance is < 3", then we consider it a match to SDSS catalog
lost_bgs['sdss_z'] = np.where(d2d < 3*u.arcsec, sdss_cat.iloc[idx]['z'], np.nan)
lost_bgs_matched = lost_bgs.loc[~np.isnan(lost_bgs['sdss_z'])]
print(f"Matched {len(lost_bgs_matched)} out of {len(lost_bgs)} lost BGS galaxies to SDSS catalog, ({len(lost_bgs_matched)/len(lost_bgs)*100:.2f}%)")

good_match = np.isclose(lost_bgs_matched.z, lost_bgs_matched.sdss_z, atol=0.001).sum()
print(f"Good match: {good_match} out of {len(lost_bgs_matched)}, or {good_match/len(lost_bgs_matched)*100:.2f}%")

## Simulations-Only Analysis

### UCHUU Issues

In [ ]:
plt.hist(all_u.all_data['M_halo'], bins=pp.Mhalo_bins, alpha=0.4)
plt.hist(all_u.all_data['uchuu_halo_mass']*10**10, bins=pp.Mhalo_bins, alpha=0.4)
plt.loglog()

# TODO do we expect the mass distribution of halos to be so different from the UCHUU SHAM catalog and our assigned halo?

In [ ]:
# TODO 1 / VMax corrections do odd thing to UCHUU Truth. Why?
pp.hod_plots(all_u)

### What effect does Fiber Assignment have on the luminosity function?

In [ ]:
pp.group_finder_centrals_halo_masses_plots(mxxl_all, [mxxl_fiberonly, mxxl_simple_4])

### Purity and Completeness

In [ ]:
pp.test_purity_and_completeness(mxxl_all, mxxl_fiberonly, mxxl_nn, mxxl_simple_4)

In [ ]:
pp.purity_complete_plots(mxxl_all, mxxl_fiberonly, mxxl_nn, mxxl_simple_4)

### Compare halos to truth

In [ ]:
pp.assigned_halo_analysis(mxxl_simple_4)

### Compare assigned implied abs mags to truth from MXXL

In [ ]:
all_unobs_counts = mxxl_all.all_data[mxxl_all.all_data.z_assigned_flag != 0].groupby('Lgal_bin').RA.count()
simple_4_ubobs_counts = mxxl_simple_4.all_data.groupby('Lgal_bin').RA.count()


In [ ]:
pp.L_func_plot([mxxl_all, mxxl_simple_4], [all_unobs_counts, simple_4_ubobs_counts])



#pp.L_func_plot([all, simple_4], [all.all_data.L_gal[all.all_data.z_assigned_flag == 0], simple_4.all_data.L_gal[simple_4.all_data.z_assigned_flag == 0]])


## SV3 Edge Effects Quantification

In [ ]:
inner_galaxies = gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_10p)
inner_galaxies.color = 'k'
inner_galaxies.name = 'SV3 Inner Galaxies'
pp.plots(inner_galaxies, bgs_sv3_simple_4_10p)

In [ ]:
fig = pp.make_map(bgs_sv3_simple_4_10p.all_data.RA.to_numpy(), bgs_sv3_simple_4_10p.all_data.Dec.to_numpy())
fig = pp.make_map(inner_galaxies.all_data.RA.to_numpy(), inner_galaxies.all_data.Dec.to_numpy(), fig=fig)

In [ ]:
bgs_sv3_simple_4_10p.all_data.groupby('Lgal_bin')['z'].median()

In [ ]:
centering_versions = [
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_10p, 1.5),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_10p, 1.4),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_10p, 1.3),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_10p, 1.2),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_10p, 1.1),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_10p, 1.0),
    gc.filter_SV3_to_avoid_edges(bgs_sv3_simple_4_10p, 0.9),
]
for i, d in enumerate(centering_versions):
    d.color = [0, i/len(centering_versions), 0]
    d.name = f'SV3 10p, {1.5-i*0.1:.1f} deg center cut'

pp.LEGENDS_ON = False
bgs_sv3_simple_4_10p.color = 'blue'
pp.fsat_by_zbins_sv3_centers(*centering_versions, z_bins=np.array([0.0, 0.03, 1.0]))

pp.LEGENDS_ON = True

pp.fsat_by_z_bins(bgs_sv3_simple_4_10p, z_bins=np.array([0.0, 0.03, 1.0]))
for d in centering_versions:
    pp.fsat_by_z_bins(d, z_bins=np.array([0.0, 0.03, 1.0]))


In [ ]:
bgs_sv3_simple_4_10p.all_data.Dec

In [ ]:
#fig = pp.make_map(bgs_sv3_simple_4_10p.all_data.RA.to_numpy(), bgs_sv3_simple_4_10p.all_data.Dec.to_numpy())

#for i, gc in enumerate(centering_versions):
#    fig = pp.make_map(gc.all_data.RA.to_numpy(), gc.all_data.Dec.to_numpy(), fig=fig)

#plot_positions(bgs_sv3_simple_4_10p, *centering_versions, tiles_df=None, split=False, DEG_LONG=7, ra_min = 186.5, dec_min = 60)
plot_positions(bgs_sv3_simple_4_10p, *centering_versions, tiles_df=None, split=False, DEG_LONG=6, ra_min = 147, dec_min = -1)

## Lost Galaxy Luminosity Function

In [ ]:
bgs_sv3_simple_4_9p.get_true_z_from(bgs_sv3_simple_4_10p.all_data)
bgs_sv3_simple_4_8p.get_true_z_from(bgs_sv3_simple_4_10p.all_data)
bgs_sv3_simple_4_7p.get_true_z_from(bgs_sv3_simple_4_10p.all_data)
bgs_sv3_simple_4_6p.get_true_z_from(bgs_sv3_simple_4_10p.all_data)
bgs_sv3_simple_4_5p.get_true_z_from(bgs_sv3_simple_4_10p.all_data)
bgs_sv3_simple_4_4p.get_true_z_from(bgs_sv3_simple_4_10p.all_data)
bgs_sv3_simple_4_3p.get_true_z_from(bgs_sv3_simple_4_10p.all_data)
bgs_sv3_simple_4_2p.get_true_z_from(bgs_sv3_simple_4_10p.all_data)
bgs_sv3_simple_4_1p.get_true_z_from(bgs_sv3_simple_4_10p.all_data)
bgs_sv3_nn_6p.get_true_z_from(bgs_sv3_simple_4_10p.all_data)

In [ ]:
def compare_L_funcs(one: pd.DataFrame, two: pd.DataFrame):
    one_counts = one.groupby('Lgal_bin').RA.count()
    two_counts = two.groupby('Lgal_bin').RA.count()
    pp.L_func_plot([one, two], [one_counts, two_counts])

In [ ]:
data = bgs_sv3_nn_6p.all_data
lostrows = data.z_assigned_flag != 0
lost_and_havetruth_rows = np.logical_and(data.z_assigned_flag != 0, data.z_T > 0)
lost_galaxies = data.loc[lostrows]
lost_withT_galaxies = data.loc[lost_and_havetruth_rows]
obs_galaxies = data.loc[~lostrows]
#.loc[obs_galaxies.z_truth > 0]

assert np.isclose(obs_galaxies.z, obs_galaxies.z_T).all()
assert np.isclose(obs_galaxies.L_gal, obs_galaxies.L_gal_T).all()

bins = np.log10(gc.L_gal_bins)

obs_counts = obs_galaxies.groupby("Lgal_bin").RA.count()
lost_assumed_counts_with_all = lost_galaxies.groupby("Lgal_bin").RA.count()
lost_assumed_counts = lost_withT_galaxies.groupby("Lgal_bin").RA.count()
lost_truth_counts = lost_withT_galaxies.groupby("Lgal_bin_T").RA.count()

print("Before Correction")
print(obs_counts.sum())
print(lost_assumed_counts_with_all.sum())
print(lost_assumed_counts.sum())
print(lost_truth_counts.sum())

# Boost them all up to the counts of obs_counts for more even comparison
lost_assumed_counts_with_all = (lost_assumed_counts_with_all * len(obs_galaxies) / len(lost_galaxies)).astype(int)
lost_assumed_counts = (lost_assumed_counts * len(obs_galaxies) / len(lost_withT_galaxies)).astype(int)
lost_truth_counts = (lost_truth_counts * len(obs_galaxies) / len(lost_withT_galaxies)).astype(int)

print("After Correction")
print(obs_counts.sum())
print(lost_assumed_counts_with_all.sum())
print(lost_assumed_counts.sum())
print(lost_truth_counts.sum())

percent_diff_with_obs = ((obs_counts - lost_assumed_counts_with_all) / lost_assumed_counts_with_all) * 100

percent_diff_with_all = ((lost_assumed_counts- lost_assumed_counts_with_all) / lost_assumed_counts_with_all) * 100

percent_diff = ((lost_assumed_counts - lost_truth_counts) / lost_truth_counts) * 100



Take a cut of SV3 whose completeness is similar to Y1 BGS.

Question: is the luminosity function of lost galaxies (that were later observed) is different from the luminosity function observed galaxies?


In [ ]:
plt.figure()
j=plt.hist(np.log10(obs_galaxies['L_gal']), bins=bins, density=True, histtype='step', color='b', label='Obs galaxies')
#j=plt.hist(np.log10(obs_galaxies['L_gal_T']), bins=bins, density=True, histtype='step', label='Obs galaxies (True)')
#j=plt.hist(np.log10(lost_galaxies['L_gal']), bins=bins, density=True, histtype='step', label='Lost galaxies')
#j=plt.hist(np.log10(lost_withT_galaxies['L_gal']), bins=bins, density=True, histtype='step', label='Lost gals w/ Truth')
j=plt.hist(np.log10(lost_withT_galaxies['L_gal_T']), bins=bins, density=True, histtype='step', color='g', label='Lost gals w/ Truth (True)')
plt.legend()
plt.title("Do Obs and Lost Gals Have Different Luminosity Funcs?")
plt.xlabel('$log(L_{gal})$')
plt.draw()

plt.figure()
obs_vs_losttruth = ((obs_counts - lost_truth_counts) / lost_truth_counts) * 100
plt.plot(obs_vs_losttruth.index, obs_vs_losttruth.values, label="Observed => Lost Truth")
plt.title("Do Obs and Lost Gals Have Different Luminosity Funcs?")
plt.xlabel("$L_{gal}$ bin")
plt.ylabel("% Change in counts")
plt.xscale('log')
plt.xlim(1E8, 1E11)
plt.ylim(-25, 25)
plt.legend()
plt.axhline(0, color='black', lw=1)
plt.draw()

They seem similar; perhaps a mild slant. Overall it seems that trying to match the observed luminosity function with the lost ones is ok.


In [ ]:
# This compares lost galaxies vs lost galaxies with truth, but using the assumed redshifts for all them. 
"""
plt.figure()
plt.plot(percent_diff_with_all.index, percent_diff_with_all.values, color='orange', label="All Lost => Lost w/ Truth")
plt.title("But...")
plt.xlabel("$L_{gal}$ bin")
plt.ylabel("% Change in counts")
plt.xscale('log')
plt.xlim(1E8, 1E11)
plt.ylim(-25, 25)
plt.legend()
plt.axhline(0, color='black', lw=1)
plt.draw()
"""

Now for lost galaxies in 6pass that we have later got redshifts for.

Question: What did our processing do to the luminosity function for lost galaxies?

In [ ]:
plt.figure()
j=plt.hist(np.log10(lost_withT_galaxies['L_gal']), bins=bins, histtype='step', color='orange', label='Lost gals w/ Truth (Assumed)')
j=plt.hist(np.log10(lost_withT_galaxies['L_gal_T']), bins=bins, histtype='step', color='g', label='Lost gals w/ Truth (True)')
plt.legend()
plt.title("Luminosity function")
plt.xlabel('$log(L_{gal})$')
plt.draw()

plt.figure()
plt.plot(percent_diff.index, percent_diff.values, color='orange', label="Truth => Assumed")
plt.title("Effect on Luminosity Function of Lost Gals")
plt.xlabel("$L_{gal}$ bin")
plt.ylabel("% Change in counts")
plt.xscale('log')
plt.xlim(1E8, 1E11)
plt.ylim(-60, 60)
plt.legend()
plt.axhline(0, color='black', lw=1)
plt.draw()

Our processing squeezes the luminosity function. We move galaxies from the wings towards the middle.

## Galaxy Neighborhood Examiner

In [ ]:
add_halo_columns(bgs_sv3_simple_4_7p)
data = bgs_sv3_simple_4_7p.all_data

In [ ]:
lost_galaxies = data.loc[data['z_assigned_flag'] != 0]
#lost_galaxies_2 = nn.all_data.loc[nn.all_data['fiber_assigned_0'] == 0]
obs_galaxies = data.loc[data['z_assigned_flag'] == 0]
#obs_galaxies_2 = nn.all_data.loc[nn.all_data['fiber_assigned_0'] == 1]
print("Lost galaxies: ", len(lost_galaxies), "Observed Galaxies: ", len(obs_galaxies))

In [ ]:
#%matplotlib qt 

In [ ]:
np.log10(gc.Mhalo_bins[16])

In [ ]:
obs_galaxies[np.logical_and(np.isclose(obs_galaxies['Mh_bin'], gc.Mhalo_bins[16]), close_enough(0.03, obs_galaxies['z']))]

In [ ]:
PLOTS_TO_MAKE = 10
GALAXY_POOL = obs_galaxies

#START_INDEX = 777
#for i in range(START_INDEX, START_INDEX + PLOTS_TO_MAKE):
#    index = lost_galaxies.index[i]
#    examine_around(index)
print("Number of galaxies to choose from: ", len(GALAXY_POOL))
indexes = np.random.randint(0, len(GALAXY_POOL)-1, size=PLOTS_TO_MAKE)
obs_galaxies.
for i in indexes:
    target = GALAXY_POOL.iloc[i]
    pp.examine_around(target, data)

### Idea: analyze entire neighborhood and look for groups of similar z galaxies, choose a z from the biggest group

In [ ]:
def neighbor_exam(target):
    nearby_angle = coord.Angle('5m')
    z_eff = target.z
    target_dist_true = z_to_ldist(target.z_obs)

    ra_max = (coord.Angle(target.RA*u.degree) + nearby_angle).value
    ra_min = (coord.Angle(target.RA*u.degree) - nearby_angle).value
    dec_max = (coord.Angle(target.Dec*u.degree) + nearby_angle).value
    dec_min = (coord.Angle(target.Dec*u.degree) - nearby_angle).value

    nearby = obs_galaxies.query('RA < @ra_max and RA > @ra_min and Dec < @dec_max and Dec > @dec_min')

    close_neighbors = 0
    if len(nearby) > 0:
        close_neighbors = np.isclose(nearby.ldist_true.to_numpy(), target_dist_true, rtol=0.0, atol=20)

    return (np.sum(close_neighbors), len(nearby), np.sum(close_neighbors)/len(nearby))

MAX = 300
close = np.empty(MAX)
total = np.empty(MAX)
frac = np.empty(MAX)
for i in range(0,MAX):
    target = lost_galaxies.iloc[i]
    close[i], total[i], frac[i] = neighbor_exam(target)

In [ ]:
max_finished = 81408
finished_close = close[0:max_finished]
finished_total = total[0:max_finished]
with open(OUTPUT_FOLDER + 'mxxl_lostgal_neighborhood_close.npy', 'wb') as f:
    np.save(f, finished_close)
with open(OUTPUT_FOLDER + 'mxxl_lostgal_neighborhood_total.npy', 'wb') as f:
    np.save(f, finished_total)

In [ ]:
with open(OUTPUT_FOLDER + 'mxxl_lostgal_neighborhood_close.npy', 'rb') as f:
    close = np.load(f)

with open(OUTPUT_FOLDER + 'mxxl_lostgal_neighborhood_total.npy', 'rb') as f:
    total = np.load(f)

frac = close / total

In [ ]:
bins = np.linspace(0,30,31)
trash = plt.hist(close, bins=bins)
plt.title("Lost Galaxies Neighbors at ~Correct z")
plt.xlabel("Count of Similar z Neighbors")
plt.ylabel("Count of Lost Galaxies")
print("Hopeless Fraction: ", np.sum(close==0) / len(close))
print("Essentially Hopeless Fraction: ", (np.sum(close==0) + np.sum(close==1)) / len(close))

In [ ]:
viable = close > 1
frac[viable]
trash=plt.hist(frac[viable], bins=30)
plt.title("Viable Lost Galaxies: Correct z Neighbor Fraction")
plt.xlabel("Fraction with Similar z")
plt.ylabel("Count of Viable Lost Galaxies")

# Tests

In [ ]:
blue_color = GLOBAL_RED_COLOR_CUT - 0.1
red_color = GLOBAL_RED_COLOR_CUT + 0.1

blue_dn = -1
red_dn = 3

results = is_quiescent_BGS_smart(np.array([7,8,9]), np.array([red_dn, np.nan, blue_dn]), np.array([blue_color, blue_color, red_color]))
assert results[0] == True
assert results[1] == False
assert results[2] == False

In [ ]:
# How many halos were assigned below a certain cutoff?
df = simple4_BGS.all_data
M_HALL_CUT = 10**11
small_halo_df = df[df.M_halo < M_HALL_CUT]

print(len(small_halo_df), len(df))

junk=plt.hist(small_halo_df.z, bins=100)
